In [1]:
#convert python txt <p1> <p2> file to json file format
import json
f = open("mapping_db_wk_clean.txt",'r',encoding='utf-8').read()
json_dict = {"values" : []}
for line in f.split("\n"):
    values = line.split(" ")
    if len(values) == 2 : db,wk = values
    json_dict["values"].append({"db" : db, "wk": wk})
with open("mapping_db_wk.json", "w") as outfile:
    json.dump(json_dict, outfile)

In [2]:
import json
#compare sameAs prop with clean db prop
with open('mapping_db_wk.json', 'r') as openfile:
    mapping_dict = json.load(openfile)

#load mapping into a dict to get constant acces time
db_sameAs_dict = {}
for i in mapping_dict["values"]:
    db_sameAs_dict[i["db"]] = i["wk"]

with open("../data/property_support.json") as json_file:
        property_dictionnary = json.load(json_file)
            


In [3]:
#filter prop count dictionary with sameAs dictionary
filtered_prop = {}
for i in db_sameAs_dict.keys():
    val = property_dictionnary.get(i)
    if val: filtered_prop[i] = val

#sort the dict
prop = dict(sorted(filtered_prop.items(), key=lambda item: item[1], reverse=True))
#keep only prop with support > 1000
prop = dict(filter(lambda x,: True if x[1] > 1000 else False, prop.items()))



In [4]:
db_sameAs_dict['<http://dbpedia.org/ontology/deathDate>']

'<http://www.wikidata.org/prop/P570>'

In [5]:
for i in prop.keys():
        print(f"{db_sameAs_dict[i]} sameAs {i} : {prop[i]}")


<http://www.wikidata.org/prop/P11143> sameAs <http://dbpedia.org/ontology/wikiPageID> : 1406259
<http://www.wikidata.org/prop/P2451> sameAs <http://dbpedia.org/ontology/birthDate> : 471625
<http://www.wikidata.org/prop/P570> sameAs <http://dbpedia.org/ontology/deathDate> : 170365
<http://www.wikidata.org/prop/P2907> sameAs <http://dbpedia.org/ontology/utcOffset> : 163232
<http://www.wikidata.org/prop/P6499> sameAs <http://dbpedia.org/ontology/populationTotal> : 154981
<http://www.wikidata.org/prop/P8497> sameAs <http://dbpedia.org/ontology/Person/height> : 111006
<http://www.wikidata.org/prop/P8483> sameAs <http://dbpedia.org/ontology/postalCode> : 106248
<http://www.wikidata.org/prop/P5286> sameAs <http://dbpedia.org/ontology/runtime> : 97818
<http://www.wikidata.org/prop/P1477> sameAs <http://dbpedia.org/ontology/originalName> : 70857
<http://www.wikidata.org/prop/P10708> sameAs <http://dbpedia.org/ontology/areaCode> : 66970
<http://www.wikidata.org/prop/P10673> sameAs <http://dbpedi

### Test values 

In [6]:
#import 
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import re

In [7]:
#const
DBPEDIA = "http://dbpedia.org/sparql"
WIKIDATA = "https://query.wikidata.org/sparql"

In [8]:
#isbn - wdt:P957

In [9]:
def sparql_query(query : str, endpoint : str ) -> dict:
    sparql = SPARQLWrapper(
        endpoint
    )
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)

    try:
        ret = sparql.queryAndConvert()
        return ret
    except Exception as e:
        print(e)
        raise Exception("Error in query")

In [10]:
def query_value_sample(prop : str) -> str:
    query = """
                select ?e ?p ?v where {
                    values ?p {"""+prop+"""}.
                    ?e ?p ?v
                } limit 100
            """
    query = re.sub(r"\n|'","",query)
    return query


In [11]:
#find prop to query
db_prop = "<http://dbpedia.org/ontology/releaseDate>"
wk_prop = "<http://www.wikidata.org/prop/statement/P10673>"
result_db = sparql_query(query_value_sample(db_prop),DBPEDIA)
result_wk = sparql_query(query_value_sample(wk_prop),WIKIDATA)

In [12]:
date_i = []
for i,j in zip(result_db["results"]["bindings"],result_wk["results"]["bindings"]):
    #print(i)
    #print(i['v']['value'],j['v']['value'])
    date_i.append(i['v']['value'])

In [35]:
hdt_query_command = "java -Xmx50G -Xms50G -jar ~soulard/QueryHDT/SparqlHomemade2.jar ~/soulard/Graphs_HDT/DBpedia/DBpedia_en.hdt ../data/P10673_releaseDate/dbpedia-releaseDate_support_query ../data/P10673_releaseDate/result.json"


['<http://dbpedia.org/ontology/releaseDate>', '<http://www.wikidata.org/prop/P10673>']
['<http://dbpedia.org/ontology/releaseDate>', '<http://www.wikidata.org/prop/P10673>']


In [ ]:
def refine_sameAs(db_wk_sameAs_file : str, wk_support_file : str) -> str:
    
    with open(db_wk_sameAs_file, 'r', encoding="UTF-8") as f:
        vars = [record for record in ijson.items(f, "head.vars")][0]

    # create a dict for each variable in our json file
    values = [set() for _ in range(len(vars))]

    with open(db_wk_sameAs_file, 'r', encoding="UTF-8") as f:
        for record in ijson.items(f, "results.bindings.item"):
            for i in range(len(vars)):
                values[i].add(f'<{record[vars[i]]["value"]}>')

    return values

In [1]:
import json
import ijson

In [35]:
def json_to_dict(json_file : str, vars : tuple[str,str]) -> dict:
    with open(json_file, encoding="UTF-8") as f:
        data = json.load(f)

    json_dict = {}
    for elem in data['results']['bindings']:
        json_dict[elem[vars[0]]["value"]] = elem[vars[1]]["value"]
    
    return json_dict

In [1]:
sameAs = "../P577_releaseDate/db-releaseDate_wk-P577_sameAs.json"
wk_support ="../P577_releaseDate/wikidata-P577.json"
sameAs_result = "../P577_releaseDate/db-releaseDate_wk-P577_sameAs_cleaned.json"
bindings = list()
var = list()
sameAs_cleaned = {}
sameAs_dict = json_to_dict(sameAs,vars=("v","e"))
wk_support_dict = json_to_dict(wk_support,vars=("e","v"))
for wk in wk_support_dict:
    if (db := sameAs_dict.get(wk)):
        sameAs_cleaned[db] = wk

with open(sameAs_result,'w',encoding="UTF-8") as f:
    json.dump(sameAs_cleaned,f)


NameError: name 'json_to_dict' is not defined

In [ ]:
    bindings = list()
    var = list()
    vars_flag = True
    for f1 in filename:
        if vars_flag:
            with open(f1, 'r', encoding = "utf-8") as f:
                f_data = json.load(f)
                var = f_data['head']['vars']
                vars_flag = False

        with open(f1, 'r', encoding = "utf-8") as f:
            f_data = json.load(f)
            bindings.extend(f_data['results']['bindings'])
            
    # Create the merged JSON object
    merged_data = {
        'head': {
            'vars': var
        },
        'results': {
            'bindings': bindings
        }